<a href="https://colab.research.google.com/github/Amanda0975/MT/blob/main/Consejeros_2023_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [125]:
import requests
import pandas as pd

df = pd.read_excel("https://github.com/Amanda0975/MT/raw/main/raw/Consejero_2023_raw.xlsx")

In [126]:
#Crear columna que suma los votos por comuna
df1=df[['Comuna','Votos']].copy()
df1=df1.groupby('Comuna').sum().reset_index().drop_duplicates()
df=pd.merge(df, df1, on="Comuna")

#Rename columnas
df=df.rename(columns={'Nro.Región': 'region_id'})
df=df.rename(columns={'Votos_x': 'Votos'})
df=df.rename(columns={'Votos_y': 'Total'})

In [127]:
#Cantidad de votos por pacto en cada comuna
df2=df[['Comuna','Votos','Pacto']].copy()
df2['Votos pacto'] = df2.groupby(['Comuna', 'Pacto'])['Votos'].transform('sum')
df=pd.merge(df, df2, on=["Comuna",'Votos','Pacto']).drop_duplicates()

##Votos derecha

In [128]:
#Checkpoint
df1=df.copy()

In [129]:
#Votos derecha
Derecha = ['CHILE SEGURO','PARTIDO REPUBLICANO DE CHILE']

# Copia del DataFrame con las columnas 'Comuna', 'Votos pacto' y 'Pacto'
df2 = df1[['Comuna', 'Votos pacto', 'Pacto']].copy()

# Filtrar las filas donde el pacto está en la lista derecha
df2 = df2[df2['Pacto'].isin(Derecha)].drop_duplicates()

#Columna nueva que agrupa por pacto derecha
df2['Votos derecha'] = df2.groupby('Comuna')['Votos pacto'].transform('sum').drop_duplicates()

In [130]:
#Database de votos de derecha por comuna
df_derecha = df2[['Comuna','Votos derecha']].copy().drop_duplicates().dropna()

##Votos PDG

In [131]:
#Checkpoint
df1=df.copy()

In [132]:
PDG = ['PARTIDO DE LA GENTE']

# Copia del DataFrame con las columnas 'Comuna', 'Votos pacto' y 'Pacto'
df2 = df1[['Comuna', 'Votos pacto', 'Pacto']].copy()

# Filtrar las filas donde el pacto está en la lista derecha y PDG
df2 = df2[df2['Pacto'].isin(PDG)]

In [133]:
#Asignar valores a columna nueva ¨Votos PDG¨
df2['Votos PDG']=df2['Votos pacto']

#Database para PDG
df_PDG=df2[['Comuna','Votos PDG']].copy().drop_duplicates().dropna()

##Merge ambos database

In [142]:
#Merge en un dataset
df=df1[['region_id','Comuna','Total']].copy().drop_duplicates()
df=pd.merge(df,df_derecha,on='Comuna',how='left').drop_duplicates()

df=pd.merge(df,df_PDG,on='Comuna',how='left').drop_duplicates()
df["Votos derecha"] = df["Votos derecha"].fillna(0)
df["Votos PDG"] = df["Votos PDG"].fillna(0)

In [145]:
#Porcentaje por categoría de voto
df['Votos derecha'] = df['Votos derecha'] / df['Total']
df['Votos PDG'] = df['Votos PDG'] / df['Total']

df=df[['region_id','Comuna',"Votos derecha","Votos PDG"]].copy().drop_duplicates()

#Exportar

In [148]:
df.to_csv('Consejeros_2023.csv', index=False)